In [2]:
import pandas as pd
import numpy as np
import pickle as pickle
import pickle
pd.options.mode.chained_assignment = None
import re

In [3]:
PATH_DATA = '/gpfs/commons/groups/gursoy_lab/aelhussein/patient_pfl/data/eicu/'
PATH_SAVE = '/gpfs/commons/groups/gursoy_lab/aelhussein/patient_pfl/20_sites/task_data/'
PATH = '/gpfs/commons/groups/gursoy_lab/aelhussein/patient_pfl/20_sites/'

## Load data

In [4]:
patient = pd.read_csv(f'{PATH_DATA}patient.csv.gz')
drugs = pd.read_csv(f'{PATH_DATA}medication.csv.gz')
diagnosis = pd.read_csv(f'{PATH_DATA}diagnosis.csv.gz')
obs = pd.read_csv(f'{PATH_DATA}physicalExam.csv.gz')

/tmp/ipykernel_211840/406109537.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  drugs = pd.read_csv(f'{PATH_DATA}medication.csv.gz')


## Features

### Physiologic

In [5]:
def processGCS(df):
    pattern = r'GCS/(\w+)\sScore/(\d+)'
    gcs_y = []
    for entry in df['physicalexampath']:
        match = re.search(pattern, entry)
        if match:
            y_value = match.group(1)
            gcs_y.append(f'GCS_{y_value}')
        else:
            gcs_y.append(f'GCS_total')
    df['physicalexamvalue'] = gcs_y
    return df

In [6]:
##Observations
obs.head()

,physicalexamid,patientunitstayid,physicalexamoffset,physicalexampath,physicalexamvalue,physicalexamtext
0,5253099,176895,9,notes/Progress Notes/Physical Exam/Physical Ex...,scored,scored
1,5253100,176895,9,notes/Progress Notes/Physical Exam/Physical Ex...,Performed - Structured,Performed - Structured
2,5253104,176895,9,notes/Progress Notes/Physical Exam/Physical Ex...,Current,68.3
3,5253105,176895,9,notes/Progress Notes/Physical Exam/Physical Ex...,Intake Total,2725
4,5253106,176895,9,notes/Progress Notes/Physical Exam/Physical Ex...,Output Total,1360


In [7]:
##Extract the physiologic data
pattern = r'^-?\d+$'
#Get GCS
gcs = obs[obs['physicalexampath'].str.contains('GCS')]
gcs_ = gcs[gcs['physicalexamtext'].apply(lambda x: bool(re.match(pattern, str(x))))]
gcs_ = processGCS(gcs_)
gcs_ = gcs_[gcs_['physicalexamvalue'] != 'GCS_total']

#Get Vitals
vs = obs[obs['physicalexampath'].str.contains('Vital Sign')]
vitals = ['HR Current', 'BP (systolic) Current', 'Resp Current', 'O2 Sat% Current']
vs_ = vs[vs['physicalexamvalue'].isin(vitals)]
vs_ = vs_[vs_['physicalexamtext'].apply(lambda x: bool(re.match(pattern, str(x))))]

#Merge
physio = pd.concat([gcs_, vs_])
physio['physicalexamtext'] = physio['physicalexamtext'].astype(int)
physio_ = physio[['patientunitstayid','physicalexamoffset', 'physicalexamvalue', 'physicalexamtext']]
physio_.rename(columns = {'physicalexamvalue':'exam', 'physicalexamtext': 'value'}, inplace = True)

In [8]:
#Filter for data in first 48 hours
MINUTES  = 48*60
physio_ = physio_[physio_['physicalexamoffset'] <= MINUTES]

In [9]:
#Get patients with full coverage
coverage = pd.DataFrame(physio_.groupby('patientunitstayid')['exam'].unique())
coverage['count'] = coverage['exam'].apply(lambda x: len(x))
full_covered = coverage[coverage['count'] == 7]
PIDS_PHYS = list(full_covered.index)

#Filter data for these patients
physio_pts = physio_[physio_['patientunitstayid'].isin(PIDS_PHYS)]

### Medications

In [10]:
drugs.head()

,medicationid,patientunitstayid,drugorderoffset,drugstartoffset,drugivadmixture,drugordercancelled,drugname,drughiclseqno,dosage,routeadmin,frequency,loadingdose,prn,drugstopoffset,gtc
0,7426715,141168,309,666,No,No,METOPROLOL TARTRATE 25 MG PO TABS,2102.0,25 3,PO,Q12H SCH,NaN,No,1826,0
1,9643232,141168,1847,1832,No,No,3 ML - IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/...,NaN,3 1,NEBULIZATION,Q4H Resp PRN,NaN,Yes,2047,0
2,10270090,141168,296,1386,No,No,ASPIRIN EC 81 MG PO TBEC,1820.0,81 3,PO,Daily,NaN,No,2390,0
3,9496768,141168,2048,2029,No,No,3 ML - IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/...,NaN,3 1,NEBULIZATION,Q4H Resp PRN,NaN,Yes,2390,0
4,11259680,141168,117,246,No,No,ENOXAPARIN SODIUM 40 MG/0.4ML SC SOLN,NaN,40 3,SC,Daily,NaN,No,1721,0


In [11]:
# Filter for drugs administered in the first 48 hours of admit
MINUTES  = 48*60
drugs = drugs[(drugs['drugordercancelled'] == 'No') & (drugs['drugstartoffset'] <= MINUTES)]
drugs_ = drugs[['patientunitstayid' ,'drugname']]
drugs_.dropna(inplace = True)

In [12]:
# Map names to integers
names = list(drugs_['drugname'].unique())
values = [i for i in range(1, len(names)+1)]
mapping = dict(zip(names, values))
mapping_back = dict(zip(values, names))
drugs_['drugid'] = drugs_['drugname'].map(mapping)
drugs_ = drugs_[['patientunitstayid', 'drugid']]

In [13]:
#Extract those with at least 4 medications
PIDS_MED = drugs_.groupby('patientunitstayid').count().loc[lambda x: x['drugid'] >= 4].index.tolist()
drugs_pts = drugs_[drugs_['patientunitstayid'].isin(PIDS_MED)]

### Diagnosis

In [14]:
def convert_to_4digit_icd9(dxStr):
	if dxStr.startswith('E') != True:
		if len(dxStr) > 5: return dxStr[:5]
		else: return dxStr
	else:
		if len(dxStr) > 6: return dxStr[:6]
		else: return dxStr

In [15]:
diagnosis.head()

,diagnosisid,patientunitstayid,activeupondischarge,diagnosisoffset,diagnosisstring,icd9code,diagnosispriority
0,4222318,141168,False,72,cardiovascular|chest pain / ASHD|coronary arte...,"414.00, I25.10",Other
1,3370568,141168,True,118,cardiovascular|ventricular disorders|cardiomyo...,NaN,Other
2,4160941,141168,False,72,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other
3,4103261,141168,True,118,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other
4,3545241,141168,True,118,cardiovascular|ventricular disorders|congestiv...,"428.0, I50.9",Other


In [16]:
#Process codes
dx_codes = diagnosis[['patientunitstayid']]
dx_codes['icd9code'] = diagnosis['icd9code'].str.split(',').str.get(0)
dx_codes.dropna(inplace = True)
dx_codes['icd9code'] = dx_codes['icd9code'].apply(convert_to_4digit_icd9)
#Filter icd9code
pattern = re.compile(r'^\D+')
dx_codes = dx_codes[dx_codes['icd9code'].str.contains(pattern) == False]

In [17]:
#Extract those with at last 4 dx
PIDS_DX = dx_codes.groupby('patientunitstayid').count().loc[lambda x: x['icd9code'] >= 3].index.tolist()
dx_pts = dx_codes[dx_codes['patientunitstayid'].isin(PIDS_DX)]

### Demographics

In [18]:
patient.head()

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
1,141178,128927,Female,52,Caucasian,60,83,NaN,162.6,08:56:00,...,Emergency Department,1,admit,54.4,54.4,09:18:00,8,Step-Down Unit (SDU),Alive,002-33870
2,141179,128927,Female,52,Caucasian,60,83,NaN,162.6,08:56:00,...,ICU to SDU,2,stepdown/other,NaN,60.4,19:20:00,2042,Home,Alive,002-33870
3,141194,128941,Male,68,Caucasian,73,92,"Sepsis, renal/UTI (including bladder)",180.3,18:18:40,...,Floor,1,admit,73.9,76.7,15:31:00,4813,Floor,Alive,002-5276
4,141196,128943,Male,71,Caucasian,67,109,NaN,162.6,20:21:00,...,ICU to SDU,2,stepdown/other,NaN,63.2,22:23:00,1463,Floor,Alive,002-37665


In [19]:
demo = patient[['patientunitstayid', 'age', 'admissionheight', 'admissionweight']]
demo.dropna(inplace = True)
PIDS_DEMO = demo['patientunitstayid'].unique().tolist()

### Patient intersection of features

In [20]:
PIDS = list(set(PIDS_PHYS).intersection(PIDS_MED).intersection(PIDS_DX).intersection(PIDS_DEMO))
print(f"number of patients with all features: {len(PIDS)}")

number of patients with all features: 44842


## Outcomes

In [21]:
patient.head()

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
1,141178,128927,Female,52,Caucasian,60,83,NaN,162.6,08:56:00,...,Emergency Department,1,admit,54.4,54.4,09:18:00,8,Step-Down Unit (SDU),Alive,002-33870
2,141179,128927,Female,52,Caucasian,60,83,NaN,162.6,08:56:00,...,ICU to SDU,2,stepdown/other,NaN,60.4,19:20:00,2042,Home,Alive,002-33870
3,141194,128941,Male,68,Caucasian,73,92,"Sepsis, renal/UTI (including bladder)",180.3,18:18:40,...,Floor,1,admit,73.9,76.7,15:31:00,4813,Floor,Alive,002-5276
4,141196,128943,Male,71,Caucasian,67,109,NaN,162.6,20:21:00,...,ICU to SDU,2,stepdown/other,NaN,63.2,22:23:00,1463,Floor,Alive,002-37665


In [22]:
patient_ = patient[patient['patientunitstayid'].isin(PIDS)]
mortality = patient_[['patientunitstayid','patienthealthsystemstayid','hospitalid', 'unitdischargestatus']]
expired = pd.get_dummies(mortality[['unitdischargestatus']])['unitdischargestatus_Expired']
mortality['expired'] = expired
mortality = mortality[['patientunitstayid', 'patienthealthsystemstayid', 'hospitalid', 'expired']]
los = patient_[['patientunitstayid', 'patienthealthsystemstayid', 'hospitalid', 'unitdischargeoffset']]

## Patient sampling

In [23]:
##select hospitals with positive cases (otherwise model fails)
def get_hospitals(unique_pt, TOTAL = 250, POS = 50):
    grouped_pt = unique_pt.groupby(['hospitalid', 'expired']).agg({'patienthealthsystemstayid': 'count'})
    pos_h = grouped_pt.loc[grouped_pt.index.get_level_values('expired') == 1][
        grouped_pt.loc[grouped_pt.index.get_level_values('expired') == 1, 'patienthealthsystemstayid'] >= POS
        ].index.get_level_values('hospitalid')
    neg_h =   grouped_pt.loc[grouped_pt.index.get_level_values('expired') == 0][
        grouped_pt.loc[grouped_pt.index.get_level_values('expired') == 0, 'patienthealthsystemstayid'] >= TOTAL - POS
        ].index.get_level_values('hospitalid')

    hospitals = list(set(neg_h).intersection(set(pos_h)))
    return hospitals

In [24]:
def sample_group(group, TOTAL = 250, POS = 50):
    expired_1 = group[group['expired'] == 1]
    expired_0 = group[group['expired'] == 0]
    n_expired_1 = POS
    n_expired_0 = TOTAL-POS
    sample_expired_1 = expired_1.sample(n=n_expired_1, random_state=1)
    sample_expired_0 = expired_0.sample(n=n_expired_0, random_state=1)
    samples = pd.concat([sample_expired_1, sample_expired_0])
    return samples.sample(frac=1, random_state=1)

In [25]:
# Select the first stay for a patient
outcome = mortality.merge(los[['patientunitstayid', 'unitdischargeoffset']], on = 'patientunitstayid')
outcome = outcome.groupby('patienthealthsystemstayid').min('unitdischargeoffset').reset_index()

In [26]:
# Filter hosptials 100 positive patients and 300 overall
unique_pt = outcome[['hospitalid','patienthealthsystemstayid', 'patientunitstayid', 'expired']]
hosp_counts = unique_pt.groupby('hospitalid').count()[['patienthealthsystemstayid']]
hospitals = get_hospitals(unique_pt)
hosp_pts = unique_pt[unique_pt['hospitalid'].isin(hospitals)]
print(f"number of hospitals: {hosp_pts['hospitalid'].nunique()}, number of patients: {hosp_pts['patientunitstayid'].nunique()}")

number of hospitals: 20, number of patients: 20221


In [27]:
# For each hospital randomly sample 300 patients
selected_patients = hosp_pts.groupby('hospitalid').apply(sample_group)
selected_patients.reset_index(level=0, drop = True, inplace = True)

## Filter datasets

In [28]:
# Filter patients in datasets
##mortality
mortality_final = mortality[mortality['patientunitstayid'].isin(selected_patients['patientunitstayid'])]
##los
los_final = los[los['patientunitstayid'].isin(selected_patients['patientunitstayid'])]
##features
demo_final = demo[demo['patientunitstayid'].isin(selected_patients['patientunitstayid'])]
dx_final = dx_pts[dx_pts['patientunitstayid'].isin(selected_patients['patientunitstayid'])]
drugs_final = drugs_pts[drugs_pts['patientunitstayid'].isin(selected_patients['patientunitstayid'])]
physio_final = physio_pts[physio_pts['patientunitstayid'].isin(selected_patients['patientunitstayid'])]

In [29]:
#Check PIDS are equal
patients_included = []
for df in [demo_final, dx_final, drugs_final, physio_final, mortality_final, los_final]:
    patients_included.append(df['patientunitstayid'].nunique())
len(set(patients_included)) == 1

True

In [30]:
#Sample sizes
sample_size = selected_patients.groupby('hospitalid')[['patientunitstayid']].count()
sample_size.columns = ['count']
sample_size.to_csv(f'{PATH_SAVE}hospitals.csv')

## Prcoess datasets for analysis

In [31]:
def add_hospid(df, hosp):
    return df.merge(hosp[['patientunitstayid', 'hospitalid']], on = 'patientunitstayid')

demo_final = add_hospid(demo_final, mortality)
dx_final = add_hospid(dx_final, mortality)
drugs_final = add_hospid(drugs_final, mortality)
physio_final = add_hospid(physio_final, mortality)

In [32]:
#Map features to integers for model
def map_codes_integers(df, column):
    names = list(df[column].unique())
    values = [i for i in range(1, len(names)+1)]
    mapping = dict(zip(names, values))
    column_new = f'{column}id'
    df[column_new] = df[column].map(mapping)
    return mapping, df.drop(columns = column)

dx_mapping, dx_final_ = map_codes_integers(dx_final, 'icd9code')
physio_mapping, physio_final_ = map_codes_integers(physio_final, 'exam')


In [33]:
#Create pivot tables
drugs_p = drugs_final[['patientunitstayid', 'drugid']].pivot_table(index='patientunitstayid', columns='drugid', aggfunc=lambda x: x['drugid'].count(), fill_value=0)
dx_p = dx_final[['patientunitstayid', 'icd9codeid']].pivot_table(index='patientunitstayid', columns='icd9codeid', aggfunc=lambda x: x['icd9codeid'].count(), fill_value=0)
physio_p = physio_final[['patientunitstayid', 'examid', 'value']].pivot_table(index='patientunitstayid', columns='examid', values='value')

## Save

In [ ]:
dataframes = [(mortality_final, 'mortality', False), 
              (los_final, 'los', False), 
              (dx_p, 'diagnosis', True), 
              (drugs_p, 'medications', True), 
              (physio_p, 'physio', True), 
              (demo_final, 'demographics', False),
              (dx_final, 'diagnosis_raw', False), 
              (drugs_final, 'medications_raw', False), 
              (physio_final, 'physio_raw', False)]
#All
for df, filename, index in dataframes:
    df.to_csv(f'{PATH_SAVE}{filename}.csv', index=index)

In [ ]:
dataframes = [(mortality_final, 'mortality', False), 
              (los_final, 'los', False), 
              (dx_p, 'diagnosis', True), 
              (drugs_p, 'medications', True), 
              (physio_p, 'physio', True), 
              (demo_final, 'demographics', False)]
#Hospital
for hosp in hospitals:
    for df, filename, index in dataframes:
        if filename in ['diagnosis', 'medications', 'physio']:
            df_ = df.merge(mortality_final[['patientunitstayid', 'hospitalid']], left_index = True, right_on = 'patientunitstayid')
            df_ = df_[df_['hospitalid'] == hosp]
            df_hosp = df_.drop('hospitalid', axis = 1).set_index('patientunitstayid')
        else:
            df_hosp = df[df['hospitalid'] == hosp]
        df_hosp.to_csv(f'{PATH_SAVE}{hosp}/{filename}.csv', index=index)

In [ ]:
with open(f'{PATH_SAVE}drug_map.pickle', 'wb') as file:
    pickle.dump(mapping, file)

with open(f'{PATH_SAVE}code_icd_map.pickle', 'wb') as file:
    pickle.dump(dx_mapping, file)

with open(f'{PATH_SAVE}physio_map.pickle', 'wb') as file:
    pickle.dump(physio_mapping, file)